<center style = "font-family: 'Lucida Console', 'Courier New', monospace;">
    <img src = "https://blogger.googleusercontent.com/img/a/AVvXsEj6-rQw5r22Bt47BUTtW5bn_dcWT7zMeADwtvsAHS3kBt6w8eWTmCM649ZcJcvosIMup6flKFIaI8p4M9ZzH1yXpEaMRjvwwfVZ_hMqgXCxtwNzEK25vTa-J2ly20by3M1zx7rTymo-tBI6Fq-mj1SJfCOXsOz0Ou1Esi4h2omvQSW98AjsONsVS-EA" width=400 height = 200>
    <h1 style = "background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%);border-radius: 20px; font-size:30px">Tensorflow - Help Protect the Great Barrier Reef 🌟🐠</h1>
</center>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Introduction</h1>
    <h2 style = "font-family: Consolas; text-align:center">Why this Competition ❓</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    Australia's stunningly beautiful Great Barrier Reef is the world’s largest coral reef and home to 1,500 species of fish, 400 species of corals, 130 species of sharks, rays, and a massive variety of other sea life.Unfortunately, the reef is under threat, in part because of the overpopulation of one particular starfish – the coral-eating crown-of-thorns starfish (or COTS for short).
    </p>
    <h2 style = "font-family: Consolas; text-align:center">Goal of Competition 🥅</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The goal of this competition is to accurately identify starfish in real-time by building an object detection model trained on underwater videos of coral reefs.
    </p>
</div>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Sponsors 💰</h1>
    <h2 style = "font-family: Consolas; text-align:center">TensorFlow</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    TensorFlow is a free and open-source software library for machine learning and artificial intelligence. It can be used across a range of tasks but has a particular focus on training and inference of deep neural networks.
    </p>
    <h2 style = "font-family: Consolas; text-align:center">CSIRO</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The Commonwealth Scientific and Industrial Research Organisation is an Australian Government agency responsible for scientific research. CSIRO works with leading organisations around the world.
    </p>
    <h2 style = "font-family: Consolas; text-align:center">GBRF</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    The Great Barrier Reef Foundation is an Australian non-profit organisation established in 1999 to help protect and preserve the Great Barrier Reef. 
    </p>
</div>

<h2 style = "font-family: Consolas">More Details</h2>
<p style = "font-family : Lucida Sans Typewriter">Check <a href = "https://www.kaggle.com/c/petfinder-pawpularity-score/data">competition page</a> for details</p>
<h2 style = "font-family : Comic Sans MS">Let's dive in ⬇️</h2>

<center><img src = "https://img.shields.io/badge/Upvote-If%20you%20found%20this%20notebook%20useful-blue" width=400 height = 400></center>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Install Libraries ⏬</h1>

In [ ]:
import torch
cuda_version_major = int(torch.version.cuda.split('.')[0])

!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh
!bash icevision_install.sh cuda11
!pip install torchtext==0.11.0 --upgrade

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Import Libraries 📚</h1>

In [ ]:
import os
import shutil

from kaggle_secrets import UserSecretsClient
import copy 
import time

import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np
import ast

import cv2

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import torch
from icevision.all import *

import wandb

from colorama import Fore, Back, Style

from IPython.display import IFrame
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Initialize Constants 🔰</h1>

In [ ]:
TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

vid0_path = "../input/tensorflow-great-barrier-reef/train_images/video_0"
vid1_path = "../input/tensorflow-great-barrier-reef/train_images/video_1"
vid2_path = "../input/tensorflow-great-barrier-reef/train_images/video_2"
vid_paths = [vid0_path, vid1_path, vid2_path]

vid0_ls = [os.path.join(vid0_path,f) for f in os.listdir(vid0_path)]
vid0_ls = sorted(vid0_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid1_ls = [os.path.join(vid1_path,f) for f in os.listdir(vid1_path)]
vid1_ls = sorted(vid1_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid2_ls = [os.path.join(vid2_path,f) for f in os.listdir(vid2_path)]
vid2_ls = sorted(vid2_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))
files_ls = [vid0_ls, vid1_ls, vid2_ls]

train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; "> Data Preprocessing ⚒️🔬</h1>

In [ ]:
def get_oldpath(x):
    if x.video_id == 0:
        path = os.path.join(vid0_path,str(x.video_frame)+".jpg")
    elif x.video_id == 1:
        path = os.path.join(vid1_path,str(x.video_frame)+".jpg")
    else:
        path = os.path.join(vid2_path,str(x.video_frame)+".jpg")
        
    return path

def get_newpath(x):
    filename = f"{x.video_id}_{x.video_frame}.jpg"
    return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [ ]:
train_df = train_df[train_df.annotations != "[]"]
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

os.makedirs("./dataset",exist_ok=True)

for i in tqdm(range(len(train_df))):
    src = train_df.iloc[i]["filepath"]
    dst = train_df.iloc[i]["newpath"]
    shutil.copy(src,dst)
    
train_df.head(2)

In [ ]:
df = train_df
df = df.explode("annotations")

df["width"] = [WIDTH]*len(df)
df["height"] = [HEIGHT]*len(df)
df["label"] = ["starfish"]*len(df)

df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)

df.loc[df["xmax"] > 1280, "xmax"] = 1280
df.loc[df["ymax"] > 720, "ymax"] = 720

df = df.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations","filepath","newpath"], axis=1)
df = df.reset_index(drop=True)
df.head(3)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Data Visualization 📊💹</h1>

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Weights and Biases</h2>
    <center><img src = "https://i.imgur.com/KISYcqD.png" width=200 height = 200></center>
    <a href = "https://wandb.ai/shanmukh/Protect%20Great%20Barrier%20Reef/runs/2eywhb67"; style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; font-size:25px">Checkout Dashboard created for this notebook</a>
    <p style = "font-family : Lucida Sans Typewriter">
    Weights and Biases is a set of Machine Learning tools used for experiment tracking, dataset versioning, and collaborating on ML projects. Weights and Biases is useful in many applications such as
    </p>  
    <ul>
          <li>Experiment Tracking</li>
          <li>Hyperparameter Tuning</li>
          <li>Data Visualization</li>
          <li>Data and model Versioning</li>
          <li>Collaborative Reports</li>
    </ul>
    <a href = "https://wandb.ai/site">Go to offocial website for more tutorials and Documentation</a>
</div>

In [ ]:
# Check https://www.kaggle.com/debarshichanda/pytorch-w-b-pawpularity-training for more details
try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('''If you want to use your W&B account, Follow these steps :
            -> go to Add-ons {Below name of notebook} -> Secrets -> Add a new Secret
            -> Label = wandb_api
            -> Value = W&B access token from https://wandb.ai/authorize 
         ''')

In [ ]:
WANDB_CONFIG = {
   "project_name" : "Protect Great Barrier Reef",
   "group_name" : "IceVision Training",
   "job_type_data" : "Data Visualization",
    "job_type_train" : "Training",
   "anonymity" : "must",
    "artifact" : "training_data"
}

run = wandb.init(
    project = WANDB_CONFIG["project_name"],
    group = WANDB_CONFIG["group_name"],
    job_type = WANDB_CONFIG["job_type_data"],
    anonymous= WANDB_CONFIG["anonymity"]
)

In [ ]:
wb_table = wandb.Table(columns = [
    "filename", "Image", "xmin", "ymin", "xmax", "ymax"
])

for i in tqdm(range(len(df))[:100]):
    row = df.loc[i]
    impath = os.path.join("./dataset",row["filename"])
    im = PIL.Image.open(impath)
    bbox = [{
            "position": {
                "minX": int(row['xmin']),
                "maxX": int(row["xmax"]),
                "minY": int(row["ymin"]),
                "maxY": int(row["ymax"])
            },
            "class_id": 1,
            "box_caption": "starfish",
            "domain": "pixel"
        }]
        
    image = wandb.Image(im,
                        boxes = {
                            "ground_truth": {
                                "box_data": bbox,
                                "class_labels" : {1: 'starfish'}
                            }
                        },
                        classes = [{"id": 0, "name": "background"}, {"id": 1, "name": "starfish"}]
                    )
    
    wb_table.add_data(row['filename'],
                      image,
                      row["xmin"],
                      row["ymin"],
                      row["xmax"],
                      row["ymax"]
                     )
    
wandb.log({'Data Visualization': wb_table})
run.finish()

In [ ]:
frame = IFrame(run.url, width=1280, height=720)
frame

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Pandas Profiling</h2>
    <center><img src = "https://pandas-profiling.github.io/pandas-profiling/docs/assets/logo_header.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. Besides, if this is not enough to convince us to use this tool, it also generates interactive reports in web format that can be presented to any person, even if they don’t know programming.
    </p>  
    <a href = "https://pandas-profiling.github.io/pandas-profiling/">Go to offocial website for documentation</a>
</div>

In [ ]:
train_report = ProfileReport(df,title="Metadata of Training images")
train_report.to_file("./train_metadata.html")
train_report

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">IceVision</h2>
    <center><img src = "https://airctic.com/0.11.0/images/icevision-logo-slogan.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    IceVision is the first agnostic computer vision framework to offer a curated collection with hundreds of high-quality pre-trained models from torchvision, MMLabs, and soon Pytorch Image Models. It orchestrates the end-to-end deep learning workflow allowing to train networks with easy-to-use robust high-performance libraries such as Pytorch-Lightning and Fastai</p> 
    IceVision Unique Features:
    <ul>
        <li>Data curation/cleaning with auto-fix</li>
        <li>Access to an exploratory data analysis dashboard</li>
        <li>Pluggable transforms for better model generalization</li>
        <li>Access to hundreds of neural net models</li>
        <li>Acccss to multiple training loop libraries</li>
        <li>Multi-task training to efficiently combine object detection, segmentation, and classification models</li>
    </ul>
    <a href = "https://airctic.com/0.11.0/">Go to offocial website for documentation</a>
</div>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Modelling 🏫</h1>

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Data Parser</h2>

In [ ]:
class PGBRParser(Parser):
    def __init__(self, template_record, data_dir, df):
        super().__init__(template_record=template_record)

        self.data_dir = data_dir
        self.df = df
        self.class_map = ClassMap(list(self.df['label'].unique()))

    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o) -> Hashable:
        return o.filename

    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(os.path.join(self.data_dir,o.filename))
            record.set_img_size(ImgSize(width=o.width, height=o.height))
            record.detection.set_class_map(self.class_map)

        record.detection.add_bboxes([BBox.from_xyxy(o.xmin, o.ymin, o.xmax, o.ymax)])
        record.detection.add_labels([o.label])

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Data Loader</h2>

In [ ]:
template_record = ObjectDetectionRecord()
parser = PGBRParser(template_record, "./dataset", df)

train_records, valid_records = parser.parse()
print(parser.class_map)

train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=512), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])

train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

samples = [train_ds[0] for _ in range(3)]
show_samples(samples, ncols=3)

In [ ]:
model_type = models.mmdet.vfnet
backbone = model_type.backbones.resnet50_fpn_mstrain_2x(pretrained=True)
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map)) 

train_dl = model_type.train_dl(train_ds, batch_size=16, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=16, num_workers=4, shuffle=False)

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Metrics</h2>

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Training</h2>

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)
learn.fine_tune(20, 0.005, freeze_epochs=1)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Inference 🔮</h1>

In [ ]:
infer_dl = model_type.infer_dl([valid_ds[0],valid_ds[4],valid_ds[7],valid_ds[9]], batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)
show_preds(preds=preds)

In [ ]:
shutil.make_archive('./checkpoints',
                    'zip',
                    './',
                    'checkpoints')

shutil.rmtree("./checkpoints")
shutil.rmtree("./dataset")
shutil.rmtree("./wandb")

<a href="./checkpoints.zip"> Download File </a>

<br>
<h3 style = "font-family: Consolas; text-align:center; color:#FF0000">If you come this far, you could've got some insights from this notebook. An upvote would be very helpful :). Kindly comment if there are any doubts or mistakes</h3>

<center><img src = "https://img.shields.io/badge/Completed-The%20End-brightgreen" width=300 height = 300></center>